In [1]:
import os
import sys
import subprocess
import matplotlib.pyplot as plt
import copy
import glob
import numpy as np
import pickle as pkl

%matplotlib inline  

In [2]:

def write_conf(conf_dir, nbase, nstrand, total_bases=10, conf_in="conf_lammps.in"): 
    '''write a new conf file that removes one base and shifts all corresponding indices'''
    
    conf_out = conf_in.replace('.in', f'_base-{nbase}_strand-{nstrand}.in')
    
    # can calculate each from total_bases
    natoms = 6*total_bases - 2
    nbonds = 6*total_bases - 4 #56
    nangles = 10*total_bases - 12
    ndihedrals = 4*total_bases - 8 
    
    atom_idx = nbase*3 - 1 + (nstrand-1)*natoms//2
    print(atom_idx)
    
    with open(conf_dir + conf_in, 'r') as file:
        
        data = file.readlines()
        new_data = []
        
        # update atom names
        nline = 0
        while nline < len(data):
            
            line = data[nline]
            
            if 'Atoms' in data[nline]:
                
                new_data.append(data[nline])
                new_data.append(data[nline+1])

                # update each required line in atoms
                for i in range(nline+2, nline+natoms+2):
                    
                    idx = int(data[i].split()[0])
                    
                    # remove atom index by not appending to new data
                    if idx == atom_idx:
                        print(idx)
                       
                    # add after changing all higher indices
                    else:

                        if idx > atom_idx:   idx -= 1
                        
                        # recombine line and write
                        line_spl = [str(idx)] + data[i].split()[1:]
                        line = '\t' + '\t'.join(line_spl) + '\n'
                        new_data.append(line)
                        
                new_data.remove(new_data[-1])
                nline += natoms+1
                
            # updates bonds    
            if 'Bonds' in data[nline]:
                
                new_data.append(data[nline])
                new_data.append(data[nline+1])
                bond_idx = 1000   # arbitrary high number to start
                
                # update each required line in Bonds
                for i in range(nline+2, nline+nbonds+2):

                    print(data[i])
                    [idx, bt, b1, b2] = [int(item) for item in data[i].split()]
                        
                    # remove atom index by not adding
                    if atom_idx == b1 or atom_idx == b2:
                        
                        # note the bond_ind and don't include in new data
                        bond_idx = idx
                        print(idx)
                    
                    # otherwise include the line with index adjustments
                    else:
                        
                        # update bonds indices to account for removal
                        if b1 > atom_idx:   b1 -= 1 
                        if b2 > atom_idx:   b2 -= 1 
                        
                        # update the line index
                        if idx > bond_idx:  idx -= 1

                        # update all indices, but keep bond type
                        line_spl = [str(idx), str(bt), str(b1), str(b2)]
                        line = '\t' + '\t'.join(line_spl) + '\n'
                        new_data.append(line)

                new_data.remove(new_data[-1])
                nline += nbonds+1
                
            # updates angles  
            if 'Angles' in data[nline]:
                
                new_data.append(data[nline])
                new_data.append(data[nline+1])
                n_removed = 0   # number of lines removed to adjust index
                
                # update each required line in Bonds
                for i in range(nline+2, nline+nangles+2):

                    print(data[i])
                    
                    # line idx, angle type and list of beads involved
                    [idx, at, a1, a2, a3] = [int(item) for item in data[i].split()]
                                            
                    # note the atom idx and don't include in new data
                    if atom_idx in [a1, a2, a3]:
                        
                        print(f'found bad angle {data[i]}')
                        n_removed += 1
                    
                    # otherwise include the value with adjustments
                    else:
                    
                        # adjust all atom indices to account for removed based:
                        if a1 > atom_idx:   a1 -= 1 
                        if a2 > atom_idx:   a2 -= 1 
                        if a3 > atom_idx:   a3 -= 1 

                        # change all higher line indices
                        idx -= n_removed

                        # reduce index of everything but the bond type
                        line_spl = [str(idx), str(at), str(a1), str(a2), str(a3)]
                        line = '\t' + '\t'.join(line_spl) + '\n'
                        new_data.append(line)
                        print(line)

                # remove repeated terminal entries at end of list
                new_data.remove(new_data[-1])
                nline += nangles+1
            
            new_data.append(line)
            nline += 1
            
            # updates dihedrals -- should be no need to change list index 
            if 'Dihedrals' in data[nline]:
                
                new_data.append(data[nline])
                new_data.append(data[nline+1])

                # update each required line in Bonds
                for i in range(nline+2, nline+ndihedrals+2):

                    print(data[i])
                    
                    # line idx, angle type and list of beads involved
                    [idx, dt, d1, d2, d3, d4] = [int(item) for item in data[i].split()]

                    # adjust all indices to account for removed based:
                    if d1 > atom_idx:   d1 -= 1 
                    if d2 > atom_idx:   d2 -= 1 
                    if d3 > atom_idx:   d3 -= 1 
                    if d4 > atom_idx:   d4 -= 1 

                    # only update reduced dihedrals
                    line_spl = [str(idx), str(dt), str(d1), str(d2), str(d3), str(d4)]
                    line = '\t' + '\t'.join(line_spl) + '\n'
                    new_data.append(line)
                    print(line)

                # remove repeated terminal entries at end of list
                new_data.remove(new_data[-1])
                nline += ndihedrals+2
            
        new_data.append(line)
        nline += 1
            
        # update each value in header categories
        new_data[2] = data[2].replace(str(natoms), str(natoms-1))
        new_data[3] = data[3].replace(str(nbonds), str(nbonds-1))
        new_data[4] = data[4].replace(str(nangles), str(nangles-n_removed))
        
    with open(conf_dir+conf_out, 'w') as file: 
        file.writelines(new_data)    
    

In [70]:
end_vals = '''            

       0 !NIMPHI

       0 !NDON

       0 !NACC

       0 !NNB
       
       '''

def write_psf(psf_dir, nbase, nstrand, total_bases=10, psf_in="in00_cvmd.psf"):
    '''write simple psf to handle removed base pairs''' 
    
    # assumes abasic conf has already been written
    conf_in=f'conf_lammps_base-{nbase}_strand-{nstrand}.in'
    
    # calculate number of atoms and index removed
    natoms = 6*total_bases - 2
    atom_idx = nbase*3 - 1 + (nstrand-1)*natoms//2
    
    # how values are arranged in psf
    bonds_per_line = 8 #atoms, 4 bonds
    angles_per_line = 9 # atoms, 3 angles
    diheds_per_line = 8 # atoms, 2 diheds
    
    # need specific spaces for VMD to parse
    s7 = '       '
    
    with open(psf_dir + psf_in, 'r') as file:
        
        spl_str = ' DNA'
        data = file.readlines()
        new_data = [data[i] for i in range(6)]
        
        # update number of atoms
        new_data[5] = new_data[5].replace(str(natoms), str(natoms-1))
        
        for i in range (6, 6+natoms):
        
            line_spl = data[i].split(spl_str)
            idx = int(line_spl[0].split()[0])
            
            # don't add to new list
            if idx == atom_idx:
                print(line_spl)
              
            # update with new line index
            else:
                if idx > atom_idx:   idx -= 1

                # recombine line and write
                line = s7 + str(idx) + spl_str + line_spl[-1]
                #line = '\t' + '\t'.join(line_spl) + '\n'
                new_data.append(line)
                
    # get rest of values from conf file
    with open(psf_dir + conf_in, 'r') as file:
        conf_lines = file.readlines()
        
        n_bonds = int(conf_lines[3].split()[0])
        n_angles = int(conf_lines[4].split()[0])
        n_diheds = int(conf_lines[5].split()[0])
        print(n_bonds, n_angles, n_diheds)
        
        for i, line in enumerate(conf_lines):
            if 'Bonds\n' == line:
                bonds_list = []
                for i in range(i+2, i+2+n_bonds):
                    bonds_list += conf_lines[i].split()[-2:]
                    
            elif 'Angles\n' == line:
                angles_list = []
                for i in range(i+2, i+2+n_angles):
                    angles_list += conf_lines[i].split()[-3:]
                    
            elif 'Dihedrals\n' == line:
                diheds_list = []
                for i in range(i+2, i+2+n_diheds):
                    diheds_list += conf_lines[i].split()[-4:]
                    
        # confirm that collected data matches total number
        print(len(bonds_list)//2, len(angles_list)//3, len(diheds_list)//4)
        print(n_bonds, n_angles, n_diheds)

    # add each category to new_data:
    new_data += ['\n', f'{s7}{n_bonds} !NBOND\n']
    num_line = []
    for i in range(2*n_bonds//bonds_per_line+1):
        min_idx = min((i+1)*bonds_per_line, len(bonds_list))
        num_line = [bonds_list[j] for j in range(i*bonds_per_line, min_idx)]
        num_line = s7 + s7.join(num_line) + '\n'
        new_data.append(num_line)
        
    new_data += ['\n', f'{s7}{n_angles} !NTHETA: angles\n']
    num_line = []
    for i in range(3*n_angles//angles_per_line+1):
        min_idx = min((i+1)*angles_per_line, len(angles_list))
        num_line = [angles_list[j] for j in range(i*angles_per_line, min_idx)]
        num_line = s7 + s7.join(num_line) + '\n'
        new_data.append(num_line)
    
    new_data += ['\n', f'{s7}{n_diheds} !NPHI: dihedrals\n']
    num_line = []
    for i in range(4*n_diheds//diheds_per_line+1):
        min_idx = min((i+1)*diheds_per_line, len(diheds_list))
        num_line = [diheds_list[j] for j in range(i*diheds_per_line, min_idx)]
        num_line = s7 + s7.join(num_line) + '\n'
        new_data.append(num_line)
        
    # remove any empty tabbed lines
    new_data = [x for x in new_data if x != f'{s7}\n']

    # write to a new file     
    psf_out = psf_in.replace('.psf', f'_base-{nbase}_strand-{nstrand}.psf')
    with open(psf_dir+psf_out, 'w+') as file: 
        file.writelines(new_data) 
        file.write(end_vals)
            

In [71]:
# specify which base position and strand

strand = 1
seq_list = glob.glob('./11bps/???????????/')
#seq_list = glob.glob('./11bps/TATATATATAT/')
total_bases = 11
#base = 5

print(seq_list)

for seq in seq_list:
    for base in range(1, 1+total_bases):

        #write_conf(f'./{seq}/', base, strand, total_bases=total_bases)
        write_psf(f'./{seq}/', base, strand, total_bases=total_bases)


['./11bps/CGCATATATAT/', './11bps/TATAGCGATAT/', './11bps/TTTTTTTTTTT/', './11bps/TATATATATAT/', './11bps/CCTATATATCC/']
['       2', '1 1    CYT  C       6   0.000000e+00   110.096           0\n']
61 96 36
61 96 36
61 96 36
['       5', '1 2    GUA  G       4   0.000000e+00   150.121           0\n']
61 94 36
61 94 36
61 94 36
['       8', '1 3    CYT  C       6   0.000000e+00   110.096           0\n']
61 94 36
61 94 36
61 94 36
['      11', '1 4    ADE  A       3   0.000000e+00   134.122           0\n']
61 94 36
61 94 36
61 94 36
['      14', '1 5    THY  T       5   0.000000e+00   125.108           0\n']
61 94 36
61 94 36
61 94 36
['      17', '1 6    ADE  A       3   0.000000e+00   134.122           0\n']
61 94 36
61 94 36
61 94 36
['      20', '1 7    THY  T       5   0.000000e+00   125.108           0\n']
61 94 36
61 94 36
61 94 36
['      23', '1 8    ADE  A       3   0.000000e+00   134.122           0\n']
61 94 36
61 94 36
61 94 36
['      26', '1 9    THY  T       5   0.000000e